In [ ]:
from os import path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from os import path
import random
from wordcloud import WordCloud, STOPWORDS

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
#nltk.download()

# Read files

In [ ]:
df_aliases = pd.read_csv('hillary-clinton-emails/Aliases.csv', index_col=0)

In [ ]:
df_aliases.head()

In [ ]:
df_email_receivers = pd.read_csv('hillary-clinton-emails/EmailReceivers.csv', index_col=0)

In [ ]:
df_email_receivers.head()

In [ ]:
df_email = pd.read_csv('hillary-clinton-emails/Emails.csv', index_col=0)

In [ ]:
df_email.head()

In [ ]:
df_persons = pd.read_csv('hillary-clinton-emails/Persons.csv', index_col=0)

In [ ]:
df_persons.head()

# Word Cloud

In [ ]:
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(60, 100)

text = ''
for value in df_email['ExtractedBodyText'].values:
    text += str(value)

mask = np.array(Image.open('logo-insa.png'))

wc = WordCloud(max_words=1000, mask=mask, margin=10, random_state=1).generate(text)

# store default colored image
default_colors = wc.to_array()
plt.title("First try")
plt.imshow(wc.recolor(color_func=grey_color_func, random_state=3))
plt.axis("off")
plt.figure()
plt.show()

# Clean text

In [ ]:
text = ''
for value in df_email['ExtractedBodyText'].values:
    text += str(value)

tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(text)

filtered_words = [word for word in tokens if word not in stopwords.words('english')]

In [ ]:
text = ''
for value in filtered_words:
    text += str(value)

wc = WordCloud(max_words=1000, margin=10, random_state=1).generate(text)

# store default colored image
default_colors = wc.to_array()
plt.title("First try")
plt.imshow(wc.recolor(color_func=grey_color_func, random_state=3))
plt.axis("off")
plt.figure()
plt.show()